In [15]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
df = pd.read_csv('data\credit_card_transactions.csv')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\chees\AppData\Local\Temp\ipykernel_18052\3307746668.py:1: SyntaxWarning: invalid escape sequence '\c'
  df = pd.read_csv('data\credit_card_transactions.csv')


In [3]:
df_dropped = df.copy()
df_dropped = df_dropped.drop(columns=['Unnamed: 0','trans_date_trans_time','cc_num','first', 'last', 'gender', 'street', 'lat', 'long', 'dob','trans_num','merch_lat', 'merch_long', 'unix_time','city', 'state'])

In [4]:
df_dropped['merch_zipcode'] = df_dropped['merch_zipcode'].fillna(df_dropped['zip'])

In [5]:
df_copy = df_dropped.copy()
df_copy = df_copy[df_copy['amt'] < 10000]

In [6]:
dummies = pd.get_dummies(df_copy['category'], drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, dummies], axis=1)
df_copy = df_copy.drop(columns=['category'])

In [7]:
scaler = StandardScaler()
df_scaled = df_copy.copy()
df_scaled[['amt', 'zip', 'city_pop', 'merch_zipcode']] = scaler.fit_transform(df_scaled[['amt', 'zip', 'city_pop', 'merch_zipcode']])
df_scaled.head()

,merchant,amt,zip,city_pop,job,is_fraud,merch_zipcode,food_dining,gas_transport,grocery_net,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel
0,"fraud_Rippin, Kub and Mann",-0.469521,-0.749142,-0.282589,"Psychologist, counselling",0,-0.754769,0,0,0,0,0,0,0,1,0,0,0,0,0
1,"fraud_Heller, Gutmann and Zieme",0.269247,1.872566,-0.293670,Special educational needs teacher,0,1.874991,0,0,0,1,0,0,0,0,0,0,0,0,0
2,fraud_Lind-Buckridge,1.084737,1.281040,-0.280406,Nature conservation officer,0,1.280622,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"fraud_Kutch, Hermiston and Farrell",-0.180328,0.402750,-0.287742,Patent attorney,0,0.399593,0,1,0,0,0,0,0,0,0,0,0,0,0
4,fraud_Keeling-Crist,-0.202290,-0.906096,-0.293835,Dance movement psychotherapist,0,-0.973533,0,0,0,0,0,0,0,0,1,0,0,0,0


In [8]:
merch_freq = df_scaled['merchant'].value_counts()
df_scaled['merchant_encoded'] = df_scaled['merchant'].apply(lambda x: merch_freq[x])
df_scaled = df_scaled.drop(columns=['merchant'])

In [9]:
job_freq = df_scaled['job'].value_counts()
df_scaled['job_encoded'] = df_scaled['job'].apply(lambda x: job_freq[x])
df_scaled = df_scaled.drop(columns=['job'])

In [10]:
df_scaled.head()

,amt,zip,city_pop,is_fraud,merch_zipcode,food_dining,gas_transport,grocery_net,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel,merchant_encoded,job_encoded
0,-0.469521,-0.749142,-0.282589,0,-0.754769,0,0,0,0,0,0,0,1,0,0,0,0,0,1267,3545
1,0.269247,1.872566,-0.293670,0,1.874991,0,0,0,1,0,0,0,0,0,0,0,0,0,2503,5099
2,1.084737,1.281040,-0.280406,0,1.280622,0,0,0,0,0,0,0,0,0,0,0,0,0,1895,511
3,-0.180328,0.402750,-0.287742,0,0.399593,0,1,0,0,0,0,0,0,0,0,0,0,0,2613,2530
4,-0.202290,-0.906096,-0.293835,0,-0.973533,0,0,0,0,0,0,0,0,1,0,0,0,0,1592,2017


In [14]:
X = df_scaled.drop(columns=['is_fraud'])
y = df_scaled['is_fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
LogReg = LogisticRegression(solver='saga',max_iter=5000)
LogReg.fit(X_train, y_train)
y_pred = LogReg.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))